# 1. BG2 tranlate Dataset Build

## (1) tlk parser

In [38]:
# tlk parser from https://github.com/3zhang/TLK-v1-file-parser-for-Python

# -*- coding: utf-8 -*-

#Author 3Zhang

import struct

b2i=lambda bstr:struct.unpack('i',bstr)[0]
b2h=lambda bstr:struct.unpack('h',bstr)[0]
i2b=lambda i:struct.pack('i',i)
h2b=lambda i:struct.pack('h',i)


#This is the version information of the header. Do not edit.
HEADER_C=b'\x54\x4c\x4b\x20\x56\x31\x20\x20\x00\x00'
COLORRED="\033[01;31m{0}\033[00m"


#a class to store each entry
class entry:
    def __init__(self,ent,n,string=None):
        self.No=n
        self.entype=b2h(ent[:2])
        self.soundinfo=ent[2:18]
        self.offset=b2i(ent[18:22])
        self.size=b2i(ent[22:])
        self.string=string
        
    def __repr__(self):
        return repr((self.No,self.entype,self.soundinfo,self.offset,self.size,self.string.decode('utf-8')))
    
    def __eq__(self, other):
        return (self.No,self.entype,self.soundinfo,self.offset,self.size,self.string) \
            == (other.No,other.entype,other.soundinfo,other.offset,other.size,other.string)


#read a tlk file. return a list of entries
def readialog(filepath):
    with open(filepath,'rb') as file:
        dg=file.read()
    header=dg[:18]
    str_o=b2i(header[14:18])
    strings=dg[str_o:]
    entries=dg[18:str_o]
    entry_l=[entries[i:i+26] for i in range(0,len(entries),26)]
    entry_l2=[entry(ent,i) for i,ent in enumerate(entry_l)]
    for i,ent in enumerate(entry_l2):
        entry_l2[i].string=strings[ent.offset:ent.offset+ent.size]
    return entry_l2
        
#Edit your dialog here. Note that string for each entry needs to be decode.
#Also, after you edit the strings, you need to encode them to binary strings.


#sort a list of entries and refresh its size and offset. You must do this after you finish editing the strings.
def refreshdialog(entryl):
    if sum([not isinstance(ent.string,bytes) for ent in entryl])>0:
        raise TypeError('String must be encoded to bytes!')
    entryl.sort(key=lambda x:x.No)
    if [ent.No for ent in entryl]!=list(range(0,len(entryl))):
        print(COLORRED.format('Warning: List index is not equal to stringref index!'))
    offset=0
    for i,ent in enumerate(entryl):
        size=len(ent.string)
        entryl[i].size=size    
        entryl[i].offset=offset if size>0 else 0
        offset+=size
        

#You must refresh the list of entries before you write them to file.
def writedialog(entryl,filepath):
    length=i2b(len(entryl))
    entries=[]
    for ent in entryl:
        entype=h2b(ent.entype)
        soundinfo=ent.soundinfo
        offset=i2b(ent.offset)
        size=i2b(ent.size)
        entb=entype+soundinfo+offset+size
        entries.append(entb)
    entries=b''.join(entries)
    soffset=i2b(18+len(entries))
    header=HEADER_C+length+soffset
    strings=[ent.string for ent in entryl]
    strings=b''.join(strings)
    dialog=header+entries+strings
    with open(filepath,'wb') as file:
        file.write(dialog)

## (2) BG2 Dialog extract

In [39]:
dial_en = readialog("./train_data/BG2(raw)/dialog_en.tlk")
dial_kr = readialog("./train_data/BG2(raw)/dialog_kr.tlk")

In [40]:
print(len(dial_kr))

dial_kr[:5]

103584


[(0, 5, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0, 9, '<NO TEXT>'),
 (1, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 9, 69, '아니오, 미안하지만, 그것들 가운데 아는 것은 없소.'),
 (2, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 78, 35, '엘민스터를 연기했습니까?'),
 (3, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 113, 117, '어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소.'),
 (4, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 230, 201, '그리고, 누가 압니까, 우리는 피코키오의 "에테르 속에서의 사흘간"을 연습하고 있었습니다. 아마도 우리는 당신들에게 한 편 보여줄 수도 있을 겁니다.')]

In [41]:
print(len(dial_kr))

dial_en[:5]

103584


[(0, 5, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0, 9, '<NO TEXT>'),
 (1, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 9, 43, "No, I'm sorry, none of them sound familiar."),
 (2, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 52, 21, 'You played Elminster?'),
 (3, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 73, 78, 'Uh, the yugoloth, was it? Yeah, you stole the show with that one, if I recall.'),
 (4, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 151, 117, 'And, who knows, we were rehearsing for Picoccio\'s "Three Days in the Ether." Perhaps we can give you a dress matinee.')]

In [42]:
dial_en[5].string

b'Oh, my dark ravens, let us stop our squawking. I shall remove this foul demeanor like a mask because, if you will still have me, I think I would quite enjoy the company of your troupe. '

In [43]:
print(dial_en[40238].string.decode('utf-8'))
print(dial_kr[40238].string.decode('utf-8'))

Outbursts like that won't get you any tips. I'll have to write Cousin Jowen and ask him to send you some pointers.
그런 식으로 분통을 터뜨리면 팁을 전혀 받지 못하지. 내 사촌 조웬에게 편지를 써서 지침을 좀 보내달라고 해야겠군.


## (3) Make DataFrame

In [44]:
import pandas as pd
import numpy as np

In [45]:
english_list = []
korean_list = []
df_dialog = pd.DataFrame(columns = ["english", "korean"])


In [46]:
for i in range(len(dial_en)):
    english_list.append(dial_en[i].string.decode('utf-8'))
    korean_list.append(dial_kr[i].string.decode('utf-8'))

df_dialog["english"] = english_list
df_dialog["korean"] = korean_list

In [47]:
df_dialog

,english,korean
0,<NO TEXT>,<NO TEXT>
1,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
2,You played Elminster?,엘민스터를 연기했습니까?
3,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
4,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
...,...,...
103579,You remind me of that mouthy gnome.,너를 보니 그 짜증 나는 노움이 떠오르는군.
103580,~ BGII:EE French Translation ~\nMoGi\nhttps://...,~ BGII:EE French Translation ~\nMoGi\nhttps://...
103581,"~ BGII:EE Volunteer Team Leader ~\nDenis ""Isay...","~ BGII:EE Volunteer Team Leader ~\nDenis ""Isay..."
103582,"~ BGII:EE Korean Translation ~\nHosub ""Teflon""\n","~ BGII:EE Korean Translation ~\nHosub ""Teflon"""


## (4) 필요없는 데이터 및 결측치 삭제

In [48]:
print(len(df_dialog['english'].iloc[103580]))
print(len(df_dialog['korean'].iloc[103580]))

60
59


### 1) 게임 대사가 아닌 내용 삭제

In [49]:
df_dialog.drop(df_dialog.index[103580:103584], axis=0, inplace=True)
df_dialog.drop(df_dialog.index[0], axis=0, inplace=True)

In [50]:
df_dialog.isnull().sum()

english    0
korean     0
dtype: int64

### 2) 해석이 안되어 있는 요소 삭제

In [51]:
df_dialog

,english,korean
1,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
2,You played Elminster?,엘민스터를 연기했습니까?
3,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
4,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
5,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...
103575,I accept this honor.,이 명예를 받아들이겠어.
103576,This place does not belong to civilized folk. ...,이 장소는 문명인들의 땅이 아니군. 조심히 걷도록 해.
103577,Cities. They reek of corruption and filth.,도시. 부패와 오물의 악취가 진동하는 장소지.
103578,Careful. This weapon is murder-sharp.,


In [52]:
print(df_dialog['english'].loc[103578])
print(len(df_dialog['korean'].loc[103578]))

Careful. This weapon is murder-sharp.
0


In [53]:
print(df_dialog['english'].loc[103578])
print(df_dialog['korean'].loc[103578])

Careful. This weapon is murder-sharp.



In [54]:
# 글자수가 너무 작은 요소 삭제(해석이 안되어 있는 대사 포함)

short_string_indexes = df_dialog.index[df_dialog.applymap(lambda x: len(x) <= 2).any(axis=1)]

print(short_string_indexes)

Index([    30,     47,     60,     82,    193,    213,    268,    369,    378,
          388,
       ...
       103560, 103562, 103563, 103564, 103566, 103567, 103568, 103572, 103573,
       103578],
      dtype='int64', length=6794)


In [55]:
df_dialog.drop(short_string_indexes, axis=0, inplace=True)

In [56]:
df_dialog

,english,korean
1,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
2,You played Elminster?,엘민스터를 연기했습니까?
3,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
4,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
5,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...
103574,Crush them!,뭉개버리자!
103575,I accept this honor.,이 명예를 받아들이겠어.
103576,This place does not belong to civilized folk. ...,이 장소는 문명인들의 땅이 아니군. 조심히 걷도록 해.
103577,Cities. They reek of corruption and filth.,도시. 부패와 오물의 악취가 진동하는 장소지.


In [57]:
print(df_dialog['english'].iloc[31])

Forget it, then. Begone from here and take your bravado with you.


In [58]:
print(len(df_dialog.korean[31]))

29


### 3) 인덱스 reset

In [59]:
df_dialog.reset_index(drop=True, inplace=True)

In [36]:
df_dialog

,english,korean
num,,
0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,You played Elminster?,엘민스터를 연기했습니까?
2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...
96780,Crush them!,뭉개버리자!
96781,I accept this honor.,이 명예를 받아들이겠어.
96782,This place does not belong to civilized folk. ...,이 장소는 문명인들의 땅이 아니군. 조심히 걷도록 해.


In [37]:
df_dialog.reset_index(inplace=True)

In [38]:
df_dialog.rename(columns={"index":"num"}, inplace=True)

In [39]:
df_dialog

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
96780,96780,Crush them!,뭉개버리자!
96781,96781,I accept this honor.,이 명예를 받아들이겠어.
96782,96782,This place does not belong to civilized folk. ...,이 장소는 문명인들의 땅이 아니군. 조심히 걷도록 해.
96783,96783,Cities. They reek of corruption and filth.,도시. 부패와 오물의 악취가 진동하는 장소지.


### 4) csv파일 저장

In [64]:
# df_dialog.to_csv(r"./train_data/BG2(raw)/dialog.csv", index=False, encoding='utf-8')

In [65]:
print(df_dialog['english'].loc[45678])
print(df_dialog['korean'].loc[45678])

We do the bidding of thieves now? Ehhh... this churns my belly like a hamster running endlessly within a wheel. This can't be a good sign, <CHARNAME>.
이젠 도둑들이 시키는 대로 하는 거야? 에에... 쳇바퀴 도는 햄스터처럼 속이 울렁거리는군. 좋은 징조가 아냐, <CHARNAME>.


In [16]:
from datasets import Dataset, DatasetDict
import pandas as pd

# df_dialog = pd.read_csv("./train_data/BG2(raw)/dialog.csv", lineterminator='\n', index_col=0)

# pandas 데이터프레임을 허깅페이스 데이텃셋으로 만들기
# https://stackoverflow.com/questions/72499850/how-to-load-two-pandas-dataframe-into-hugginfaces-dataset-object

In [17]:
print(df_dialog['english'].loc[93711])
print(df_dialog['korean'].loc[93711])

Shapechange: Iron Golem

Strength: 24
Dexterity: 9

Base Armor Class: 3
Number of Attacks: 1
Attack Damage: 4d10 (crushing)

Special Abilities:
 – Fire Resistance: 125%
 – Cold Resistance: 100%
 – Electrical Resistance: 100%
 – Acid Resistance: 100%
 – Magic Resistance: 100%
 – Magic Fire Resistance: 125%
 – Magic Cold Resistance: 100%
 – Slashing Resistance: 20%
 – Crushing Resistance: 20%
 – Piercing Resistance: 20%
 – Missile Resistance: 20%
 – Immune to backstab, weapons of +2 enchantment or less, sleep, fear, hold, charm, paralysis, poison.
변신: 철 골렘

힘: 24
민첩: 9

기본 방어도: 3
라운드당 공격 횟수: 1
공격 피해: 4d10 (타격)

특수 능력:
- 화염 저항: 125%
- 냉기 저항: 100%
- 전기 저항: 100%
- 산성 저항: 100%
- 마법 저항: 100%
- 마법 화염 저항: 125%
- 마법 냉기 저항: 100%
- 베기 저항: 20%
- 타격 저항: 20%
- 관통 저항: 20%
- 원거리 저항: 20%
- 배후 공격, +2 이하 무기, 수면, 공포, 속박, 현혹, 마비, 독에 면역.


## (5) 허깅페이스 데이터셋 만들기

### 1) 판다스 데이터프레임에서 허깅페이스 데이터셋으로 변환

In [67]:
datasets_train = DatasetDict({
    "train": Dataset.from_pandas(df_dialog)
})

In [68]:
datasets_train

DatasetDict({
    train: Dataset({
        features: ['num', 'english', 'korean'],
        num_rows: 96785
    })
})

In [69]:
datasets_train["train"][93711]

{'num': 93711,
 'english': 'Shapechange: Iron Golem\n\nStrength: 24\nDexterity: 9\n\nBase Armor Class: 3\nNumber of Attacks: 1\nAttack Damage: 4d10 (crushing)\n\nSpecial Abilities:\n – Fire Resistance: 125%\n – Cold Resistance: 100%\n – Electrical Resistance: 100%\n – Acid Resistance: 100%\n – Magic Resistance: 100%\n – Magic Fire Resistance: 125%\n – Magic Cold Resistance: 100%\n – Slashing Resistance: 20%\n – Crushing Resistance: 20%\n – Piercing Resistance: 20%\n – Missile Resistance: 20%\n – Immune to backstab, weapons of +2 enchantment or less, sleep, fear, hold, charm, paralysis, poison.',
 'korean': '변신: 철 골렘\n\n힘: 24\n민첩: 9\n\n기본 방어도: 3\n라운드당 공격 횟수: 1\n공격 피해: 4d10 (타격)\n\n특수 능력:\n- 화염 저항: 125%\n- 냉기 저항: 100%\n- 전기 저항: 100%\n- 산성 저항: 100%\n- 마법 저항: 100%\n- 마법 화염 저항: 125%\n- 마법 냉기 저항: 100%\n- 베기 저항: 20%\n- 타격 저항: 20%\n- 관통 저항: 20%\n- 원거리 저항: 20%\n- 배후 공격, +2 이하 무기, 수면, 공포, 속박, 현혹, 마비, 독에 면역.'}

### 2) 허깅페이스 로그인 및 허브에 데이터셋 업로드

In [70]:
from huggingface_hub import login

# login(token="your_token")

# datasets_train.push_to_hub("aeolian83/DnD_translate")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/97 [00:00<?, ?ba/s]

# 2. PoE2(Pillars of Eternity2) translate dataset build

## (1) xml parser를 통한 game script 추출

### 1) 대상폴더 리스트 확정

In [1]:
# xml parser import

import xml.etree.ElementTree as ET

In [4]:
ko_folder_list = ['./train_data/POE2(raw)/lax1_exported/localized/ko',
 './train_data/POE2(raw)/lax2_exported/localized/ko',
 './train_data/POE2(raw)/lax3_exported/localized/ko',
 './train_data/POE2(raw)/laxa_exported/localized/ko',
 './train_data/POE2(raw)/laxb_exported/localized/ko',
 './train_data/POE2(raw)/laxc_exported/localized/ko',
 './train_data/POE2(raw)/laxd_exported/localized/ko',
 './train_data/POE2(raw)/laxf_exported/localized/ko',
 './train_data/POE2(raw)/laxg_exported/localized/ko',
 './train_data/POE2(raw)/laxh_exported/localized/ko',
 './train_data/POE2(raw)/laxi_exported/localized/ko']


en_folder_list = ['./train_data/POE2(raw)/lax1_exported/localized/en',
 './train_data/POE2(raw)/lax2_exported/localized/en',
 './train_data/POE2(raw)/lax3_exported/localized/en',
 './train_data/POE2(raw)/laxa_exported/localized/en',
 './train_data/POE2(raw)/laxb_exported/localized/en',
 './train_data/POE2(raw)/laxc_exported/localized/en',
 './train_data/POE2(raw)/laxd_exported/localized/en',
 './train_data/POE2(raw)/laxf_exported/localized/en',
 './train_data/POE2(raw)/laxg_exported/localized/en',
 './train_data/POE2(raw)/laxh_exported/localized/en',
 './train_data/POE2(raw)/laxi_exported/localized/en']

### 2) 대상 파일리스트 작성

In [6]:
import os

def get_all_xml_files_in_subdirectories(root_folder):
    xml_file_list = []

    # 재귀적으로 폴더 내부를 탐색하며 XML 파일 리스트 추출
    for root, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".stringtable"):
                file_path = os.path.join(root, file)
                xml_file_list.append(file_path)

    return xml_file_list

# 시작 폴더 설정
starting_folder = "./train_data/POE2(raw)/lax1_exported/localized/en"  # 실제 시작 폴더 경로로 변경해주세요

# 시작 폴더부터 모든 XML 파일을 리스트로 추출
# all_xml_files = get_all_xml_files_in_subdirectories(starting_folder)

In [7]:
# en_folder_list
# ko_folder_list

all_en_filelist = []
all_ko_filelist = []

for i in range(len(en_folder_list)):
    en_xmlfiles = get_all_xml_files_in_subdirectories(en_folder_list[i])
    ko_xmlfiles = get_all_xml_files_in_subdirectories(ko_folder_list[i])
    all_en_filelist += en_xmlfiles
    all_ko_filelist += ko_xmlfiles

print(len(all_en_filelist))
print(len(all_ko_filelist))

412
412


### 3) xml file parsing

In [8]:
# # XML 파일을 파싱하여 ElementTree 객체 생성
# tree = ET.parse(script_dir)
# root = tree.getroot()

# # 'DefaultText' 태그 안의 텍스트 추출
# default_texts = []
# for default_text_elem in root.iter('DefaultText'):
#     default_texts.append(default_text_elem.text)
print(len(all_en_filelist))
print(len(all_ko_filelist))


print("START!!!")
all_en_default_texts = []
all_ko_default_texts = []
for i in range(len(all_en_filelist)):
    tree = ET.parse(all_en_filelist[i])
    root = tree.getroot()

    for default_text_elem in root.iter('DefaultText'):
        all_en_default_texts.append(default_text_elem.text)

for i in range(len(all_en_filelist)):
    tree = ET.parse(all_ko_filelist[i])
    root = tree.getroot()

    for default_text_elem in root.iter('DefaultText'):
        all_ko_default_texts.append(default_text_elem.text)

print("DONE!!!")

print()
print()

print(len(all_en_default_texts))
print(len(all_ko_default_texts))

412
412
START!!!
DONE!!!


18952
18952


In [9]:
all_en_default_texts

['"Konstanten, how does it look on me? Pretty good, ac?"',
 '"Agracima, how you dote on me, captain." ',
 '"Fall in line, Konstanten. Do not make me tell you twice." ',
 '"I\'ll show you how it\'s done, Konstanten." ',
 '"Konstanten! Don\'t you dare!" ',
 '"Konstanten, stop wasting time and put your skills to use over here." ',
 '"Ask the dwarf. I am not made for this." ',
 '"Did you see that, Konstanten?! Ha!"',
 '"At least let the dwarf keep me company?" ',
 '"Konstanten, get over here with your massage oils." ',
 '"Try and stay quiet, Konstanten. You\'re a noisy boy." ',
 '"Hope Fassina didn\'t see that."',
 '"I bet Fassina could do it."',
 '"That one\'s for Fassina!"',
 '"Wonder what Fassina\'s up to."',
 '"Hey Fassina, you heard the one about- ah, never mind."',
 '"You think Fassina\'ll like this?"',
 '"Gotta look good for the ladies."',
 '"How\'s the view, Fassina?"',
 '"Fassina, please! Get back up!"',
 '"Don\'t move, Fassina! I\'ll be there soon!"',
 '"Grouchy wizard\'s down!"'

In [10]:
all_ko_default_texts

['"Konstanten, how does it look on me? Pretty good, ac?"',
 '"Agracima, how you dote on me, captain."',
 '"Fall in line, Konstanten. Do not make me tell you twice."',
 '"I\'ll show you how it\'s done, Konstanten."',
 '"Konstanten! Don\'t you dare!"',
 '"Konstanten, stop wasting time and put your skills to use over here."',
 '"Ask the dwarf. I am not made for this."',
 '"Did you see that, Konstanten?! Ha!"',
 '"At least let the dwarf keep me company?"',
 '"Konstanten, get over here with your massage oils."',
 '"Try and stay quiet, Konstanten. You\'re a noisy boy."',
 '"Hope Fassina didn\'t see that."',
 '"I bet Fassina could do it."',
 '"That one\'s for Fassina!"',
 '"Wonder what Fassina\'s up to."',
 '"Hey Fassina, you heard the one about- ah, never mind."',
 '"You think Fassina\'ll like this?"',
 '"Gotta look good for the ladies."',
 '"How\'s the view, Fassina?"',
 '"Fassina, please! Get back up!"',
 '"Don\'t move, Fassina! I\'ll be there soon!"',
 '"Grouchy wizard\'s down!"',
 '"We\'

## (2) Data 정제

### 1) 번역이 안되어 있는 데이터 확인

In [14]:
# 한글 script와 영문 script가 일치하는 부분 확인해서 번역이 안되어 있는 문장 확인

print((all_ko_default_texts[0]) == (all_en_default_texts[0]))

True


In [13]:
print(len(all_ko_default_texts))
print(len(all_en_default_texts))

double_list =[]
for i, (elem1, elem2) in enumerate(zip(all_en_default_texts, all_ko_default_texts)):
    if not elem1 == elem2:
        continue
    double_list.append(i)

print("겹치는 문장개수:", len(double_list))
# print(double_list)

18952
18952
겹치는 문장개수: 200


### 2) 데이터가 같은 말뭉치 각각의 리스트에서 제거

In [15]:
print(len(double_list))
print(len(all_ko_default_texts))
print(len(all_en_default_texts))

clean_all_en_default_texts = []
clean_all_ko_default_texts = []

for elem1, elem2 in zip(all_en_default_texts, all_ko_default_texts):
    if elem1 is not None and elem2 is not None:
        if elem1 != elem2:
            clean_all_en_default_texts.append(elem1)
            clean_all_ko_default_texts.append(elem2)
        
print("Done")
print(len(clean_all_en_default_texts))
print(len(clean_all_ko_default_texts))

200
18952
18952
Done
18612
18612


### 3) DataFrame

In [18]:
# df_dialog
df_dialog_poe = pd.DataFrame(columns = ["english", "korean"])

In [19]:
df_dialog_poe['english'] = clean_all_en_default_texts
df_dialog_poe['korean'] = clean_all_ko_default_texts

In [20]:
df_dialog_poe

,english,korean
0,"""Agracima, how you dote on me, captain.""","""Agracima, how you dote on me, captain."""
1,"""Fall in line, Konstanten. Do not make me tell...","""Fall in line, Konstanten. Do not make me tell..."
2,"""I'll show you how it's done, Konstanten.""","""I'll show you how it's done, Konstanten."""
3,"""Konstanten! Don't you dare!""","""Konstanten! Don't you dare!"""
4,"""Konstanten, stop wasting time and put your sk...","""Konstanten, stop wasting time and put your sk..."
...,...,...
18607,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
18608,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
18609,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
18610,Quest Failed.,퀘스트에 실패했습니다.


In [25]:
# 글자수가 너무 작은 요소 삭제(해석이 안되어 있는 대사 포함)

# short_string_indexes = df_dialog_poe.index[df_dialog_poe.applymap(lambda x: len(x) <= 1).any(axis=1)]

# print(short_string_indexes)

Index([5484, 16941], dtype='int64')


In [27]:
# print(df_dialog_poe['english'].loc[16941])
# print(df_dialog_poe['korean'].loc[16941])

Book
책


In [29]:
df_dialog_poe.reset_index(inplace=True)

In [31]:
df_dialog_poe.rename(columns={"index":"num"}, inplace=True)

In [32]:
df_dialog_poe

,num,english,korean
0,0,"""Agracima, how you dote on me, captain.""","""Agracima, how you dote on me, captain."""
1,1,"""Fall in line, Konstanten. Do not make me tell...","""Fall in line, Konstanten. Do not make me tell..."
2,2,"""I'll show you how it's done, Konstanten.""","""I'll show you how it's done, Konstanten."""
3,3,"""Konstanten! Don't you dare!""","""Konstanten! Don't you dare!"""
4,4,"""Konstanten, stop wasting time and put your sk...","""Konstanten, stop wasting time and put your sk..."
...,...,...,...
18607,18607,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
18608,18608,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
18609,18609,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
18610,18610,Quest Failed.,퀘스트에 실패했습니다.


## (3) Dataset Build

### 1) DataFrame Concat

In [58]:
df_dialog_all = pd.concat([df_dialog, df_dialog_poe])

In [59]:
df_dialog_all

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
18607,18607,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
18608,18608,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
18609,18609,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
18610,18610,Quest Failed.,퀘스트에 실패했습니다.


In [60]:
df_dialog_all.reset_index(inplace=True)

In [61]:
df_dialog_all

,index,num,english,korean
0,0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...,...
115392,18607,18607,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
115393,18608,18608,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
115394,18609,18609,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
115395,18610,18610,Quest Failed.,퀘스트에 실패했습니다.


In [62]:
df_dialog_all.drop(["num"], axis=1, inplace=True)
df_dialog_all.drop(["index"], axis=1, inplace=True)

In [63]:
df_dialog_all.reset_index(inplace=True)

In [64]:
df_dialog_all

,index,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
115392,115392,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
115393,115393,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
115394,115394,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
115395,115395,Quest Failed.,퀘스트에 실패했습니다.


In [65]:
df_dialog_all.rename(columns={"index":"num"}, inplace=True)

In [66]:
df_dialog_all


,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
115392,115392,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
115393,115393,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
115394,115394,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
115395,115395,Quest Failed.,퀘스트에 실패했습니다.


### 2) Dataset Build

In [67]:
datasets_train = DatasetDict({
    "train": Dataset.from_pandas(df_dialog_all)
})

In [68]:
datasets_train

DatasetDict({
    train: Dataset({
        features: ['num', 'english', 'korean'],
        num_rows: 115397
    })
})

### 3) HuggingFace Upload

In [69]:
from huggingface_hub import login

login(token="your_token")

datasets_train.push_to_hub("aeolian83/DnD_translate_v1.5")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/116 [00:00<?, ?ba/s]